# 📋 레시피 데이터 처리 및 통합

이 노트북에서는 식품안전나라 API에서 레시피 데이터를 수집하고,
여러 개의 레시피 순서 컬럼을 하나로 통합하여 정리합니다.

## 🎯 목표
1. 식품안전나라 API에서 레시피 데이터 수집
2. MANUAL01~MANUAL20 컬럼을 하나의 'recipe_steps' 컬럼으로 통합
3. 레시피 순서를 '1. 씻는다, 2. 손질한다, 3. 썬다' 형태로 정리
4. 깔끔한 CSV 파일로 저장

## 1. 📊 데이터 수집 및 기본 처리

In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import re
import requests
import json
import os
from time import sleep

In [2]:
def fetch_all_recipe_data():
    """
    식품안전나라 API에서 모든 레시피 데이터를 수집하는 함수
    """
    API_KEY = "8434c606f68343abaeb9"
    SERVICE_ID = "COOKRCP01"
    DATA_TYPE = "json"
    BATCH_SIZE = 1000
    
    all_recipes = []
    start_idx = 1
    
    print("🔄 API에서 레시피 데이터 수집을 시작합니다...")
    
    while True:
        end_idx = start_idx + BATCH_SIZE - 1
        url = f"http://openapi.foodsafetykorea.go.kr/api/{API_KEY}/{SERVICE_ID}/{DATA_TYPE}/{start_idx}/{end_idx}"
        
        print(f"📥 요청 중: {start_idx} ~ {end_idx}")
        
        try:
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                
                if 'COOKRCP01' in data:
                    if start_idx == 1:
                        total_count = data['COOKRCP01']['total_count']
                        print(f"📊 전체 데이터 개수: {total_count}개")
                    
                    if 'row' in data['COOKRCP01']:
                        recipes = data['COOKRCP01']['row']
                        all_recipes.extend(recipes)
                        print(f"✅ 현재까지 수집된 데이터: {len(all_recipes)}개")
                        
                        if len(recipes) < BATCH_SIZE:
                            print("🎉 마지막 배치 완료")
                            break
                    else:
                        print("⚠️ 더 이상 데이터가 없습니다.")
                        break
                else:
                    print("❌ 예상하지 못한 응답 구조:", data)
                    break
            else:
                print(f"❌ API 요청 실패: {response.status_code}")
                break
                
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            break
        
        start_idx += BATCH_SIZE
        sleep(0.1)  # API 호출 간 대기
    
    print(f"\n🎯 총 {len(all_recipes)}개의 레시피 데이터 수집 완료!")
    return all_recipes

# 데이터 수집 실행
recipes_data = fetch_all_recipe_data()

🔄 API에서 레시피 데이터 수집을 시작합니다...
📥 요청 중: 1 ~ 1000
📊 전체 데이터 개수: 1136개
✅ 현재까지 수집된 데이터: 1000개
📥 요청 중: 1001 ~ 2000
✅ 현재까지 수집된 데이터: 1136개
🎉 마지막 배치 완료

🎯 총 1136개의 레시피 데이터 수집 완료!


In [3]:
# DataFrame 생성 및 기본 정보 확인
if recipes_data:
    df = pd.DataFrame(recipes_data)
    
    print("📋 DataFrame 생성 완료!")
    print(f"📏 데이터 크기: {df.shape}")
    print(f"📊 총 컬럼 수: {len(df.columns)}개")
    
    # 현재 컬럼 순서 확인
    print(f"\n🔍 현재 컬럼 순서 (처음 10개): {list(df.columns)[:10]}")
    
    # 주요 컬럼 미리보기
    key_columns = ['RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_ENG']
    print(f"\n🔍 주요 데이터 미리보기:")
    print(df[key_columns].head())
    
    # 레시피 단계 컬럼 확인
    manual_cols = [col for col in df.columns if col.startswith('MANUAL') and not col.startswith('MANUAL_IMG')]
    print(f"\n📝 레시피 단계 컬럼 ({len(manual_cols)}개): {manual_cols[:6]}...")
    
else:
    print("❌ 수집된 데이터가 없습니다.")
    df = pd.DataFrame()

📋 DataFrame 생성 완료!
📏 데이터 크기: (1136, 55)
📊 총 컬럼 수: 55개

🔍 현재 컬럼 순서 (처음 10개): ['RCP_PARTS_DTLS', 'RCP_WAY2', 'MANUAL_IMG20', 'MANUAL20', 'RCP_SEQ', 'INFO_NA', 'INFO_WGT', 'INFO_PRO', 'MANUAL_IMG13', 'MANUAL_IMG14']

🔍 주요 데이터 미리보기:
           RCP_NM RCP_WAY2 RCP_PAT2 INFO_ENG
0       새우 두부 계란찜       찌기       반찬      220
1        부추 콩가루 찜       찌기       반찬      215
2       방울토마토 소박이       기타       반찬       45
3  순두부 사과 소스 오이무침       기타       반찬       75
4       사과 새우 북엇국      끓이기     국&찌개       65

📝 레시피 단계 컬럼 (20개): ['MANUAL20', 'MANUAL01', 'MANUAL08', 'MANUAL09', 'MANUAL06', 'MANUAL07']...


In [4]:
# DataFrame 컬럼 순서 정리
def reorder_dataframe_columns(df):
    """
    DataFrame의 컬럼을 논리적 순서로 재정렬하는 함수
    
    Args:
        df: 원본 DataFrame
        
    Returns:
        컬럼이 재정렬된 DataFrame
    """
    if df.empty:
        return df
    
    # 1. 기본 정보 컬럼들 (순서대로)
    basic_info = [
        'RCP_SEQ',        # 레시피 순번
        'RCP_NM',         # 레시피명
        'RCP_WAY2',       # 조리방법
        'RCP_PAT2',       # 요리종류
        'INFO_WGT',       # 중량
        'INFO_ENG',       # 칼로리
        'INFO_CAR',       # 탄수화물
        'INFO_PRO',       # 단백질
        'INFO_FAT',       # 지방
        'INFO_NA'         # 나트륨
    ]
    
    # 2. 재료 및 설명 컬럼들
    content_info = [
        'RCP_PARTS_DTLS', # 재료정보
        'RCP_NA_TIP',     # 나트륨 팁
        'HASH_TAG'        # 해시태그
    ]
    
    # 3. 이미지 컬럼들 (메인 이미지)
    main_images = [
        'ATT_FILE_NO_MAIN',  # 메인 이미지
        'ATT_FILE_NO_MK'     # 만들기 이미지
    ]
    
    # 4. 레시피 단계별 컬럼들 (MANUAL01~MANUAL20 + 해당 이미지)
    manual_columns = []
    for i in range(1, 21):
        manual_col = f'MANUAL{i:02d}'
        img_col = f'MANUAL_IMG{i:02d}'
        
        if manual_col in df.columns:
            manual_columns.append(manual_col)
        if img_col in df.columns:
            manual_columns.append(img_col)
    
    # 5. 기타 컬럼들 (위에서 분류되지 않은 것들)
    all_categorized = set(basic_info + content_info + main_images + manual_columns)
    other_columns = [col for col in df.columns if col not in all_categorized]
    
    # 최종 컬럼 순서 결정
    ordered_columns = []
    
    # 존재하는 컬럼만 추가
    ordered_columns.extend([col for col in basic_info if col in df.columns])
    ordered_columns.extend([col for col in content_info if col in df.columns])
    ordered_columns.extend([col for col in main_images if col in df.columns])
    ordered_columns.extend(other_columns)  # 기타 컬럼들
    ordered_columns.extend(manual_columns)  # 레시피 단계는 마지막에
    
    return df[ordered_columns]

# 컬럼 순서 정리 실행
if not df.empty:
    print("🔄 컬럼 순서를 논리적으로 정리하는 중...")
    print(f"정리 전 컬럼 순서 (처음 10개): {list(df.columns)[:10]}")
    
    df_reordered = reorder_dataframe_columns(df)
    
    print(f"✅ 컬럼 순서 정리 완료!")
    print(f"정리 후 컬럼 순서 (처음 15개): {list(df_reordered.columns)[:15]}")
    
    # 컬럼 카테고리별 개수 확인
    basic_cols = ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA']
    content_cols = ['RCP_PARTS_DTLS', 'RCP_NA_TIP', 'HASH_TAG']
    manual_cols = [col for col in df_reordered.columns if col.startswith('MANUAL')]
    
    print(f"\n📊 컬럼 구성:")
    print(f"- 기본정보: {len([c for c in basic_cols if c in df_reordered.columns])}개")
    print(f"- 내용정보: {len([c for c in content_cols if c in df_reordered.columns])}개") 
    print(f"- 레시피단계: {len(manual_cols)}개")
    print(f"- 전체: {len(df_reordered.columns)}개")
    
    # 원본 df 업데이트
    df = df_reordered
    
else:
    print("❌ DataFrame이 없습니다.")

🔄 컬럼 순서를 논리적으로 정리하는 중...
정리 전 컬럼 순서 (처음 10개): ['RCP_PARTS_DTLS', 'RCP_WAY2', 'MANUAL_IMG20', 'MANUAL20', 'RCP_SEQ', 'INFO_NA', 'INFO_WGT', 'INFO_PRO', 'MANUAL_IMG13', 'MANUAL_IMG14']
✅ 컬럼 순서 정리 완료!
정리 후 컬럼 순서 (처음 15개): ['RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 'INFO_WGT', 'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA', 'RCP_PARTS_DTLS', 'RCP_NA_TIP', 'HASH_TAG', 'ATT_FILE_NO_MAIN', 'ATT_FILE_NO_MK']

📊 컬럼 구성:
- 기본정보: 10개
- 내용정보: 3개
- 레시피단계: 40개
- 전체: 55개


## 2. 🔧 레시피 순서 통합 처리

In [5]:
# 빈 값들을 null로 치환하여 데이터 정리
def clean_empty_values(df):
    """
    DataFrame의 빈 문자열, 공백만 있는 값들을 null(NaN)로 치환
    """
    df_cleaned = df.copy()
    
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == 'object':  # 문자열 컬럼만 처리
            # 빈 문자열을 NaN으로 치환
            df_cleaned[col] = df_cleaned[col].replace('', np.nan)
            # 공백만 있는 문자열을 NaN으로 치환
            df_cleaned[col] = df_cleaned[col].replace(r'^\s*$', np.nan, regex=True)
    
    return df_cleaned

if not df.empty:
    print("🧹 데이터 정리 중...")
    df_cleaned = clean_empty_values(df)
    df = df_cleaned
    print("✅ 빈 값들을 null로 정리 완료!")
else:
    print("❌ DataFrame이 없습니다.")

🧹 데이터 정리 중...
✅ 빈 값들을 null로 정리 완료!


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33220\2353208295.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned[col] = df_cleaned[col].replace('', np.nan)


In [6]:
def combine_recipe_steps(df):
    """
    MANUAL01~MANUAL20 컬럼들을 하나의 'recipe_steps' 컬럼으로 통합하는 함수
    
    Args:
        df: 원본 DataFrame
        
    Returns:
        통합된 recipe_steps 컬럼이 추가된 DataFrame
    """
    df_combined = df.copy()
    recipe_steps = []
    
    print("🔄 레시피 순서 통합 처리 중...")
    
    for idx in range(len(df)):
        steps = []
        step_number = 1
        
        # MANUAL01부터 MANUAL20까지 순서대로 확인
        for i in range(1, 21):
            manual_col = f'MANUAL{i:02d}'
            
            if manual_col in df.columns:
                step_text = df.iloc[idx][manual_col]
                
                # null이 아니고 의미있는 텍스트가 있는 경우만 추가
                if pd.notna(step_text) and str(step_text).strip():
                    # 기존 번호 제거 및 텍스트 정리
                    clean_text = str(step_text).strip()
                    # 앞의 숫자와 점 제거 (예: "1. ", "2. ")
                    clean_text = re.sub(r'^\d+\.\s*', '', clean_text)
                    # 뒤의 알파벳 제거 (예: "a", "b", "c")
                    clean_text = re.sub(r'[a-zA-Z]$', '', clean_text).strip()
                    
                    if clean_text:  # 정리 후에도 텍스트가 있으면
                        steps.append(f"{step_number}. {clean_text}")
                        step_number += 1
        
        # 모든 단계를 하나의 문자열로 합치기 (줄바꿈으로 구분)
        if steps:
            recipe_steps.append('\n'.join(steps))
        else:
            recipe_steps.append('')  # 빈 문자열
    
    # 새 컬럼 추가
    df_combined['recipe_steps'] = recipe_steps
    
    print(f"✅ recipe_steps 컬럼 생성 완료!")
    return df_combined

# 레시피 순서 통합 실행
if not df.empty:
    df_with_steps = combine_recipe_steps(df)
    
    # 결과 확인
    print(f"\n📋 첫 번째 레시피 예시 ({df_with_steps.iloc[0]['RCP_NM']}):")
    print("=" * 60)
    print(df_with_steps.iloc[0]['recipe_steps'])
    
    # 통계 정보
    total_recipes = len(df_with_steps)
    recipes_with_steps = (df_with_steps['recipe_steps'] != '').sum()
    print(f"\n📊 통계:")
    print(f"- 전체 레시피: {total_recipes:,}개")
    print(f"- 조리순서가 있는 레시피: {recipes_with_steps:,}개")
    print(f"- 조리순서 비율: {(recipes_with_steps/total_recipes)*100:.1f}%")
    
    # 원본 df 업데이트
    df = df_with_steps
    
else:
    print("❌ DataFrame이 없습니다.")

🔄 레시피 순서 통합 처리 중...
✅ recipe_steps 컬럼 생성 완료!

📋 첫 번째 레시피 예시 (새우 두부 계란찜):
1. 손질된 새우를 끓는 물에 데쳐 건진다.
2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)를 함께 섞어 그릇에 담는다.
3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분 정도 찐다.

📊 통계:
- 전체 레시피: 1,136개
- 조리순서가 있는 레시피: 1,135개
- 조리순서 비율: 99.9%


In [7]:
# 레시피 없는 요리 제거
df=df[df['recipe_steps']!=''].reset_index(drop=True)
print(len(df))

1135


In [8]:
df

,RCP_SEQ,RCP_NM,RCP_WAY2,RCP_PAT2,INFO_WGT,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,INFO_NA,...,MANUAL_IMG16,MANUAL17,MANUAL_IMG17,MANUAL18,MANUAL_IMG18,MANUAL19,MANUAL_IMG19,MANUAL20,MANUAL_IMG20,recipe_steps
0,28,새우 두부 계란찜,찌기,반찬,NaN,220,3,14,17,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 손질된 새우를 끓는 물에 데쳐 건진다.\n2. 연두부, 달걀, 생크림, 설탕에..."
1,29,부추 콩가루 찜,찌기,반찬,NaN,215,20,14,9,240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣..."
2,31,방울토마토 소박이,기타,반찬,NaN,45,9,2,1,277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1. 물기를 빼고 2cm 정도의 크기로 썰은 부추와 양파를 양념장에 섞어 양념속을 ...
3,32,순두부 사과 소스 오이무침,기타,반찬,NaN,75,10,4,2,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 사과, 순두부를 믹서에 넣고 곱게 갈아 소스를 만든다.\n2. 오이는 소금으로..."
4,33,사과 새우 북엇국,끓이기,국&찌개,NaN,65,2,12,1,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 북어채를 잘게 손으로 찢어 찬물에 헹구어 준비한 후 양파, 표고버섯, 사과는 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,3292,양파토마토스튜,끓이기,국&찌개,288,36.2,4.9,1.6,1.2,155.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1. 물(8컵)에 육수 재료와 마늘(2쪽)을 넣어 끓으면 다시마를 건진 뒤 약한 불...
1131,3294,황금팽이 비빔국수,기타,기타,176.2,102.7,6.9,12.8,2.6,314.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 황금팽이버섯은 밑동을 자른 뒤 낱낱이 가르고, 양파, 상추, 깻잎, 오이는 채..."
1132,3295,맛살 미역줄기전,굽기,반찬,70.6,128.2,11.9,4.1,7.2,519.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 미역 줄기는 흐르는 물에 여러 번 헹구고, 찬물에 1시간 정도 담가둔다.\n2..."
1133,3297,해산물샐러드와 미나리소스,볶기,반찬,192.8,216.9,8.6,23.9,9.6,454.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1. 새송이버섯은 모양대로 도톰하게 썰고, 방울토마토는 꼭지를 제거한 뒤 열십자 모..."


In [9]:
if not df.empty and 'recipe_steps' in df.columns:
    # 주요 컬럼들만 선택해서 저장
    selected_columns = [
        'RCP_SEQ', 'RCP_NM', 'RCP_WAY2', 'RCP_PAT2', 
        'INFO_ENG', 'INFO_CAR', 'INFO_PRO', 'INFO_FAT', 'INFO_NA',
        'RCP_PARTS_DTLS', 'recipe_steps', 'RCP_NA_TIP', 'HASH_TAG'
    ]
    
    # 존재하는 컬럼만 필터링
    available_columns = [col for col in selected_columns if col in df.columns]
    df_final = df[available_columns]

In [10]:
## 3. 🔧 재료 정보 파싱 (fix_json_formatting.py 참고)

def clean_text(text):
    """
    텍스트에서 불필요한 기호들 제거
    """
    if not text:
        return text
    
    # 불필요한 기호들 제거
    unwanted_symbols = ['●', '◆', '◇', '■', '□', '▲', '△', '▼', '▽', '★', '☆', '※', '∙', '•']
    
    for symbol in unwanted_symbols:
        text = text.replace(symbol, '')
    
    # 연속된 공백을 하나로 정리
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def parse_ingredient_text(ingredient_text):
    """
    재료 텍스트를 파싱하여 구조화된 정보 추출
    """
    if not ingredient_text or not ingredient_text.strip():
        return None

    # 불필요한 기호 제거
    ingredient_text = clean_text(ingredient_text.strip())

    result = {
        'name': '',
        'amount': '',
        'unit': '',
        'description': ''
    }

    # 괄호 안의 설명 추출
    description_match = re.search(r'\(([^)]+)\)', ingredient_text)
    if description_match:
        result['description'] = clean_text(description_match.group(1))
        ingredient_text = ingredient_text.replace(description_match.group(0), '').strip()

    # 숫자와 단위 추출
    amount_match = re.search(r'([0-9./]+)\s*([a-zA-Z가-힣]+)', ingredient_text)
    if amount_match:
        result['amount'] = amount_match.group(1)
        result['unit'] = amount_match.group(2)
        result['name'] = clean_text(ingredient_text[:amount_match.start()].strip())
    else:
        result['name'] = clean_text(ingredient_text)

    return result

def structure_ingredients(rcp_parts_dtls):
    """
    RCP_PARTS_DTLS 텍스트를 구조화된 재료 정보로 변환
    """
    if pd.isna(rcp_parts_dtls) or not str(rcp_parts_dtls).strip():
        return {'categories': []}

    # 전체 텍스트에서 불필요한 기호 제거
    clean_text_content = clean_text(str(rcp_parts_dtls))
    lines = clean_text_content.split('\n')
    
    categories = []
    current_category = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if ',' in line:
            ingredients = []
            for ingredient in line.split(','):
                parsed = parse_ingredient_text(ingredient.strip())
                if parsed and parsed['name']:
                    ingredients.append(parsed)

            if current_category:
                current_category['ingredients'].extend(ingredients)
            else:
                categories.append({
                    'category': '기본재료',
                    'ingredients': ingredients
                })
        else:
            parsed = parse_ingredient_text(line)
            if parsed and parsed['amount']:
                if current_category:
                    current_category['ingredients'].append(parsed)
                else:
                    categories.append({
                        'category': '기본재료',
                        'ingredients': [parsed]
                    })
            else:
                # 카테고리명도 정리
                clean_category_name = clean_text(line)
                if clean_category_name:  # 정리 후에도 텍스트가 있으면
                    current_category = {
                        'category': clean_category_name,
                        'ingredients': []
                    }
                    categories.append(current_category)

    return {'categories': categories}

# RCP_PARTS_DTLS 컬럼을 구조화된 재료 정보로 직접 파싱하여 업데이트
print("🔄 재료 정보 파싱 중 (불필요한 기호 제거 포함)...")

parsed_ingredients = []
for idx, row in df_final.iterrows():
    try:
        structured = structure_ingredients(row['RCP_PARTS_DTLS'])
        # Python 딕셔너리 문자열로 변환 (작은따옴표 사용)
        dict_str = str(structured).replace('"', "'")
        parsed_ingredients.append(dict_str)
        
        if (idx + 1) % 100 == 0:
            print(f"진행상황: {idx + 1}/{len(df_final)} 완료 ({((idx + 1)/len(df_final)*100):.1f}%)")
        
    except Exception as e:
        print(f"인덱스 {idx}에서 오류: {e}")
        parsed_ingredients.append("{'categories':[]}")

# RCP_PARTS_DTLS 컬럼을 파싱된 내용으로 직접 대체
df_final['RCP_PARTS_DTLS'] = parsed_ingredients

print("✅ 재료 정보 파싱 완료 (불필요한 기호 제거됨)!")
print(f"📊 파싱된 레시피 수: {len(df_final):,}개")

🔄 재료 정보 파싱 중 (불필요한 기호 제거 포함)...
진행상황: 100/1135 완료 (8.8%)
진행상황: 200/1135 완료 (17.6%)
진행상황: 300/1135 완료 (26.4%)
진행상황: 400/1135 완료 (35.2%)
진행상황: 500/1135 완료 (44.1%)
진행상황: 600/1135 완료 (52.9%)
진행상황: 700/1135 완료 (61.7%)
진행상황: 800/1135 완료 (70.5%)
진행상황: 900/1135 완료 (79.3%)
진행상황: 1000/1135 완료 (88.1%)
진행상황: 1100/1135 완료 (96.9%)
✅ 재료 정보 파싱 완료 (불필요한 기호 제거됨)!
📊 파싱된 레시피 수: 1,135개


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33220\32712803.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['RCP_PARTS_DTLS'] = parsed_ingredients


In [11]:
# 파싱 결과 확인
print("🔍 파싱 결과 미리보기:")
print("=" * 60)

# 첫 번째 레시피의 파싱된 재료 정보 확인
first_recipe = df_final.iloc[0]
print(f"📋 레시피명: {first_recipe['RCP_NM']}")
print(f"📦 파싱된 재료 정보:")
print(first_recipe['RCP_PARTS_DTLS'][:500] + "...")

# JSON 파싱하여 구조 확인
try:
    import json
    # 작은따옴표를 큰따옴표로 변환하여 JSON 파싱
    json_str = first_recipe['RCP_PARTS_DTLS'].replace("'", '"')
    parsed_sample = json.loads(json_str)
    print(f"\n✅ JSON 파싱 성공!")
    print(f"📊 구조: {len(parsed_sample['categories'])}개 카테고리")
    
    for i, cat in enumerate(parsed_sample['categories']):
        print(f"  {i+1}. {cat['category']}: {len(cat['ingredients'])}개 재료")
        # 첫 번째 카테고리의 재료들 일부 출력
        if i == 0 and cat['ingredients']:
            for j, ingredient in enumerate(cat['ingredients'][:3]):  # 처음 3개만
                print(f"     - {ingredient['name']} {ingredient['amount']} {ingredient['unit']} ({ingredient['description']})")
            if len(cat['ingredients']) > 3:
                print(f"     ... 외 {len(cat['ingredients'])-3}개")
                
except Exception as e:
    print(f"❌ JSON 파싱 실패: {e}")

# 전체 통계
print(f"\n📈 전체 통계:")
print(f"- 총 레시피 수: {len(df_final):,}개")
print(f"- RCP_PARTS_DTLS 컬럼 파싱 완료")
print(f"- 데이터 형태: 구조화된 Python 딕셔너리 문자열")

🔍 파싱 결과 미리보기:
📋 레시피명: 새우 두부 계란찜
📦 파싱된 재료 정보:
{'categories': [{'category': '기본재료', 'ingredients': [{'name': '새우두부계란찜 연두부', 'amount': '75', 'unit': 'g', 'description': '3/4모'}, {'name': '칵테일새우', 'amount': '20', 'unit': 'g', 'description': '5마리'}, {'name': '달걀', 'amount': '30', 'unit': 'g', 'description': '1/2개'}, {'name': '생크림', 'amount': '13', 'unit': 'g', 'description': '1큰술'}, {'name': '설탕', 'amount': '5', 'unit': 'g', 'description': '1작은술'}, {'name': '무염버터', 'amount': '5', 'unit': 'g', 'description': '1작은술'}]}]}...

✅ JSON 파싱 성공!
📊 구조: 1개 카테고리
  1. 기본재료: 6개 재료
     - 새우두부계란찜 연두부 75 g (3/4모)
     - 칵테일새우 20 g (5마리)
     - 달걀 30 g (1/2개)
     ... 외 3개

📈 전체 통계:
- 총 레시피 수: 1,135개
- RCP_PARTS_DTLS 컬럼 파싱 완료
- 데이터 형태: 구조화된 Python 딕셔너리 문자열


In [12]:
# RCP_PARTS_DTLS 컬럼을 문자열에서 딕셔너리로 변환
import ast

def convert_string_to_dict(string_dict):
    """문자열로 된 딕셔너리를 실제 딕셔너리로 변환"""
    try:
        return ast.literal_eval(string_dict)
    except:
        # 파싱 실패시 빈 딕셔너리 반환
        return {'categories': []}

print("🔄 RCP_PARTS_DTLS를 문자열에서 딕셔너리로 변환 중...")

# 변환 실행
df_final['RCP_PARTS_DTLS'] = df_final['RCP_PARTS_DTLS'].apply(convert_string_to_dict)

print("✅ 변환 완료!")

# 결과 확인
print(f"\n📊 변환 후 데이터 타입: {type(df_final.iloc[0]['RCP_PARTS_DTLS'])}")
print(f"📋 첫 번째 레시피 재료 정보:")
first_ingredients = df_final.iloc[0]['RCP_PARTS_DTLS']
print(f"- 카테고리 개수: {len(first_ingredients['categories'])}")
print(f"- 첫 번째 카테고리: {first_ingredients['categories'][0]['category']}")
print(f"- 재료 개수: {len(first_ingredients['categories'][0]['ingredients'])}")
print(f"- 첫 번째 재료: {first_ingredients['categories'][0]['ingredients'][0]}")

🔄 RCP_PARTS_DTLS를 문자열에서 딕셔너리로 변환 중...
✅ 변환 완료!

📊 변환 후 데이터 타입: <class 'dict'>
📋 첫 번째 레시피 재료 정보:
- 카테고리 개수: 1
- 첫 번째 카테고리: 기본재료
- 재료 개수: 6
- 첫 번째 재료: {'name': '새우두부계란찜 연두부', 'amount': '75', 'unit': 'g', 'description': '3/4모'}


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33220\44469340.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['RCP_PARTS_DTLS'] = df_final['RCP_PARTS_DTLS'].apply(convert_string_to_dict)


In [13]:
lists = ['INFO_ENG','INFO_CAR','INFO_PRO','INFO_FAT','INFO_NA']
df_final[lists] = df_final[lists].astype(float)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33220\3431943205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[lists] = df_final[lists].astype(float)


In [24]:
df_final.to_csv('rcp.csv', index=False)

In [15]:
df_final.head()

,RCP_SEQ,RCP_NM,RCP_WAY2,RCP_PAT2,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,INFO_NA,RCP_PARTS_DTLS,recipe_steps,RCP_NA_TIP,HASH_TAG
0,28,새우 두부 계란찜,찌기,반찬,220.0,3.0,14.0,17.0,99.0,"{'categories': [{'category': '기본재료', 'ingredie...","1. 손질된 새우를 끓는 물에 데쳐 건진다.\n2. 연두부, 달걀, 생크림, 설탕에...","나트륨의 배출을 도와주는 것으로 알려진 칼륨이 풍부한 시금치와 소금, 간장 등의 양...",연두부
1,29,부추 콩가루 찜,찌기,반찬,215.0,20.0,14.0,9.0,240.0,"{'categories': [{'category': '기본재료', 'ingredie...","1. 부추는 깨끗이 씻어 물기를 제거하고, 5cm 길이로 썰고 부추에 날콩가루를 넣...",콩가루로 버무려 감칠맛과 고소한 맛으로 나트륨 사용량을 줄여보세요. 부추는 피를 맑...,날콩가루
2,31,방울토마토 소박이,기타,반찬,45.0,9.0,2.0,1.0,277.0,"{'categories': [{'category': '기본재료', 'ingredie...",1. 물기를 빼고 2cm 정도의 크기로 썰은 부추와 양파를 양념장에 섞어 양념속을 ...,소금에 절이는 오이 대신 방울토마토를 사용하여 나트륨 섭취를 줄였어요. 토마토에는 ...,방울토마토
3,32,순두부 사과 소스 오이무침,기타,반찬,75.0,10.0,4.0,2.0,22.0,"{'categories': [{'category': '기본재료', 'ingredie...","1. 사과, 순두부를 믹서에 넣고 곱게 갈아 소스를 만든다.\n2. 오이는 소금으로...",사과에는 팩틴이 풍부하여 체내 나쁜 콜레스테롤을 감소시키고 나트륨 배출을 촉진시켜줘...,순두부
4,33,사과 새우 북엇국,끓이기,국&찌개,65.0,2.0,12.0,1.0,78.0,"{'categories': [{'category': '기본재료', 'ingredie...","1. 북어채를 잘게 손으로 찢어 찬물에 헹구어 준비한 후 양파, 표고버섯, 사과는 ...",소금과 간장 대신 북어채와 새우의 짠맛으로 간을 한 담백한 맛의 북엇국을 만들었어요...,NaN


In [16]:
df_final.describe()

,INFO_ENG,INFO_CAR,INFO_PRO,INFO_FAT,INFO_NA
count,1135.000000,1135.000000,1135.000000,1135.000000,1135.000000
mean,235.251559,26.843947,12.921991,9.002026,224.520018
std,163.251707,63.767661,29.457958,17.932328,220.352795
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,112.000000,8.000000,4.000000,2.195000,63.500000
50%,202.300000,16.600000,9.200000,5.600000,168.800000
75%,322.650000,34.620000,15.650000,11.800000,313.950000
max,1311.000000,1838.000000,807.000000,505.000000,2441.000000


In [25]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135 entries, 0 to 1134
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RCP_SEQ         1135 non-null   object 
 1   RCP_NM          1135 non-null   object 
 2   RCP_WAY2        1135 non-null   object 
 3   RCP_PAT2        1135 non-null   object 
 4   INFO_ENG        1135 non-null   float64
 5   INFO_CAR        1135 non-null   float64
 6   INFO_PRO        1135 non-null   float64
 7   INFO_FAT        1135 non-null   float64
 8   INFO_NA         1135 non-null   float64
 9   RCP_PARTS_DTLS  1135 non-null   object 
 10  recipe_steps    1135 non-null   object 
 11  RCP_NA_TIP      1081 non-null   object 
 12  HASH_TAG        357 non-null    object 
dtypes: float64(5), object(8)
memory usage: 115.4+ KB


In [27]:
df_final['HASH_TAG'].value_counts()

HASH_TAG
가슴살     35
닭가슴살    19
등심      16
저염간장    15
다짐육     13
        ..
아욱       1
유부       1
토마토      1
미소된장     1
홍합       1
Name: count, Length: 152, dtype: int64